# Student Loan Risk with Deep Learning

In [25]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [26]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [27]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [28]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [29]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [30]:
# Define the target set y using the credit_ranking column
y =  loans_df["credit_ranking"].values
# Display a sample of y
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [31]:
# Define features set X by selecting all columns but credit_ranking
x = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
x

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [32]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state= 1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [33]:
# Create a StandardScaler instance
scale_model = StandardScaler()

# Fit the scaler to the features training dataset
scale_model.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scale_model.transform(X_train)
X_test_scaled = scale_model.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [34]:
# Define the the number of inputs (features) to the model
inputs = len(x.columns)

# Review the number of features
inputs

11

In [35]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_layer = 6

# Define the number of hidden nodes for the second hidden layer
second_hidden_layer = 6

# Define the number of neurons in the output layer
output_layer = 1

In [36]:
# Create the Sequential model instance
NN = tf.keras.models.Sequential()

# Add the first hidden layer
NN.add(tf.keras.layers.Dense(units=first_hidden_layer, input_dim=inputs, activation='relu'))

# Add the second hidden layer
NN.add(tf.keras.layers.Dense(units=second_hidden_layer, input_dim=inputs, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
NN.add(tf.keras.layers.Dense(units=output_layer, input_dim = inputs, activation= 'sigmoid'))

c:\Users\Matt\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Display the Sequential model summary
NN.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [38]:
# Compile the Sequential model
NN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Fit the model using 50 epochs and the training data
fit_network = NN.fit(X_train_scaled,Y_train, epochs=50)

Epoch 1/50


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 939us/step - accuracy: 0.6347 - loss: 0.6757 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.6937 - loss: 0.6489
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7206 - loss: 0.6413
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.7436 - loss: 0.6130
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7383 - loss: 0.6049
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.7273 - loss: 0.5873
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.7464 - loss: 0.5710
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.7472 - loss: 0.5632
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.7487 - loss: 0.5456
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7452 - loss: 0.5366 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.7595 - loss: 0.5360
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [40]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = NN.evaluate(X_test_scaled, Y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 8ms/step - accuracy: 0.7325 - loss: 0.5266
Loss: 0.5266138911247253, Accuracy: 0.7325000166893005


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [42]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
NN.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [43]:
# Set the model's file path
file_path

# Load the model to a new object
NN_reloaded = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [47]:
# Make predictions with the test data
NN_predictions = NN_reloaded.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
NN_predictions[:10]

13/13 - 0s - 3ms/step


array([[0.31883454],
       [0.3189091 ],
       [0.86301416],
       [0.6979698 ],
       [0.96881396],
       [0.69440633],
       [0.8796885 ],
       [0.3157314 ],
       [0.5712636 ],
       [0.25505665]], dtype=float32)

In [50]:
# Save the predictions to a DataFrame and round the predictions to binary results
NN_df = pd.DataFrame(columns=["prediction"], data= NN_predictions)
NN_df["prediction"] = round(NN_df["prediction"], 0)
NN_df

,prediction
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [51]:
# Print the classification report with the y test data and predictions
print(classification_report(Y_test, NN_df))

              precision    recall  f1-score   support

           0       0.69      0.78      0.73       188
           1       0.78      0.69      0.73       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.74      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
      
        Creating a recommendation system that guides students in selecting the most suitable student loan options is a crucial endeavor in today's educational landscape. To effectively recommend loan programs, it's essential to consider a comprehensive dataset that begins with the student's chosen major. Given that some academic programs are more intensive, potentially extending the time required to graduate, these students may need larger loans.

        Key pieces of data to enhance the recommendation system include:

        1. Whether the student is studying in-state or out-of-state, as this affects tuition rates.
        2. The presence of any scholarships, which can significantly reduce the need for loans.
        3. The student's income level, which impacts their ability to repay the loan.
        4. Marital status, as it may influence financial responsibilities and eligibility for different loan terms.
        5. Current market trends and economic factors, which affect loan interest rates and repayment terms.
        6. Behavioral data such as risk tolerance, which helps predict which loan options a student might prefer based on their comfort with financial risk.
        
        Together, these parameters are crucial in shaping a student's decision-making process regarding whether to take out a loan. By analyzing these factors, a recommendation system can provide tailored, sensible loan options that align with individual financial situations and academic trajectories.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

        I recommend developing the model using context-aware filtering. This approach is advantageous over content-based filtering, which often relies on extensive historical user data that may not always be available. Context-aware filtering also has a distinct edge over collaborative filtering, which primarily depends on past user actions. These historical dependencies can limit the adaptability of recommendations in response to shifting current conditions.

        Context-aware filtering excels by incorporating real-time data, such as prevailing economic trends and individual financial situations, like a student's current income. This ability to integrate dynamic contextual information makes it exceptionally suited for recommending student loans, ensuring that the advice remains relevant, timely, and tailored to each student's unique circumstances. By focusing on the present context rather than solely on historical patterns, this method offers a more robust framework for guiding students towards the most suitable loan options.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
  
        One of the foremost challenges in designing a student loan recommendation system today is the volatility of the market. Inflation and interest rates are currently at exceptionally high levels and show no signs of stabilizing in the near future. This unpredictability, especially with policies at the federal level, complicates the modeling of loan repayment conditions, as these policies can significantly influence the terms and feasibility of repaying loans.

        Additionally, accurately gathering data such as an individual's risk tolerance poses a substantial challenge. This metric is inherently subjective and can be unreliable in survey contexts, where respondents might misrepresent their actual preferences in an attempt to appear more resilient or attractive as loan candidates. Furthermore, a student's major, which can significantly influence the duration of their academic program, often changes during their college career, potentially derailing the predictive accuracy of the system. These factors together make it particularly challenging to develop a system that reliably anticipates the needs and capacities of students regarding loan options.
